In [1]:
## Packages -----
cran_packages <- list(
    "Rcpp",
    "data.table",
    "microbenchmark",
    "ggplot2"
)
invisible(
    lapply(
        cran_packages,
        function(i) {
            if (!require(i, character.only = T)) {
                install.packages(i)
                library(i, character.only = T)
            } else {
                library(i, character.only = T)
            }
        }
    )
)
# Package containing a set of helper functions:
if (!require(fromScratchR)) {
    devtools::install_github("pat-alt/fromScratchR")
    library(fromScratchR)
} else {
    library(fromScratchR)
}

Loading required package: Rcpp

Loading required package: data.table

Loading required package: microbenchmark

Loading required package: ggplot2

Loading required package: fromScratchR



# Multi-level regression

## Model and synthetic data

We consider a multilevel regression model with group level predictors (varying intercepts) as in Gelman, Hill (2007). For the first level we have for individual $i\in[1,N]$:

$$
\begin{aligned}
&& y_i &= z_{j[i]} + \mathbf{X}_i^T \beta + \varepsilon_i, && \varepsilon_i \sim \mathcal{N}(0, \phi)
\end{aligned}
$$

where $\mathbf{X}_i$ is a $(K \times 1)$ vector of individual-level predictors. Consequently we have that 

$$
\begin{aligned}
&& y_i|z_j[i], \mathbf{X}_i &\sim \mathcal{N}(z_{j[i]} + \mathbf{X}_i^T \beta, \phi)
\end{aligned}
$$

where $\phi^{-1}$ can be thought of as the precision of the data. 

For the second level we have for group $j\in[1,M]$

$$
\begin{aligned}
&& z_j &= \mathbf{u}_j^T \gamma + \eta_j, && \eta_j \sim \mathcal{N}(0, \psi)
\end{aligned}
$$

where $z_j$ are unobserved and $\mathbf{u}_j$ is $(L \times 1)$ vector of observed group-level predictors.

Consequently we have that 

$$
\begin{aligned}
&& z_j|\mathbf{u}_j&\sim \mathcal{N}(\mathbf{u}_j^T \gamma, \psi)
\end{aligned}
$$

where $\psi^{-1}$ can be thought of as the precision of the latent factors. To simulate data from this model we can use the helper function below. For now we will look at a single set of default parameter choices. Note that the choices for `phi` and `psi` roughly imply that the latent, group-level information is twice as precise as the individual level data. We would therefore expect the algorithm to produce final posterior densities of the latent variables that are are more similar to their estimated prior density then the likelihood of the observed data given the latent variables. 

In [3]:
syn_multi_level_reg <- function(
  N=10000,
  M=1000,
  beta=c(1,0.1),
  phi=0.1,
  a=1,
  b=0.5,
  psi=0.05,
  seed=123
) {
  set.seed(seed)
  # 1.) Latent factors: ----
  u <- matrix(rnorm(M))
  z <- a + b * u + rnorm(n=nrow(u),sd=sqrt(psi))
  # Group lengths:
  wgts <- runif(M)
  wgts <- wgts/sum(wgts)
  n_j <- pmax(round(wgts * N),10)
  to_allocate <- N - sum(n_j)
  while (to_allocate!=0) {
    if (to_allocate > 0) {
      idx <- sample(M,1)
      n_j[idx] <- n_j[idx] + 1
      to_allocate <- to_allocate - 1
    } else {
      idx <- sample(M,1)
      n_j[idx] <- n_j[idx] - 1
      to_allocate <- to_allocate + 1
    }
  }
  Z <- rep(0, N)
  from <- 1
  for (j in 1:M) {
    idx <- from:(from+n_j[j]-1)
    Z[idx] <- z[j]
    from <- from + n_j[j]
  }
  # 2.) Output: ----
  K <- length(beta)
  X <- matrix(rnorm(N*K), nrow=N)
  y <- Z + X %*% beta + rnorm(n=nrow(X),sd=sqrt(phi))
  model <- list(
    data = list(
      X = X,
      y = y,
      u = u,
      n_j = n_j
    ),
    z = z,
    M = M,
    N = N,
    params = list(
      beta = beta,
      phi = phi,
      psi = psi,
      a = a,
      b = b
    )
  )
  return(model)
}

syn_data <- syn_multi_level_reg()

## EM algorithm

We will use the EM algorithm the uncover the latent factors and fit the multilevel regression model. To do so we first need to derive the *expected* complete data log-likelihood (CLL):

$$
\begin{aligned}
&& Q(\theta, \theta_{t-1}) &= \mathbb{E}_{p(\mathbf{z}|\mathbf{y}, \mathbf{X}, \theta_{t-1})} \left[ \log \ell (\theta | \mathbf{y}, \mathbf{X}, \mathbf{z}) \right]
\end{aligned}
$$

Below we provide a high-level overview of the involved mathematics. Derivations can be found in the appendix if necessary.

### E-step

Generally it can be difficult to compute the expected CLL, but here it is fairly straight-forward to compute the intergral over $\mathbf{z}$ since the posterior density $p(\mathbf{z}|\mathbf{y}, \mathbf{X}, \theta_{t-1})$ is mathematically tractable due to conjugacy. Specifically we have for the posterior density

$$
\begin{aligned}
&& z_j|\mathbf{y}_j,\mathbf{X}_j, \theta&\sim \mathcal{N}(\mu_j, v_j) \\
&& v_j &= \frac{1}{\frac{n_j}{\phi}+\frac{1}{\psi}} \\
&& \mu_j &= \frac{\frac{n_j}{\phi} (\bar{\mathbf{y}}_j - \bar{\mathbf{X}}_j \beta)}{\frac{n_j}{\phi}+\frac{1}{\psi}} + \frac{\frac{1}{\psi} (\mathbf{u}^T_j \gamma)}{\frac{n_j}{\phi}+\frac{1}{\psi}} \\
\end{aligned}
$$

and hence for the expected CLL

$$
\begin{aligned}
&& Q(\theta, \theta_{t-1})&= - {1\over{2}} \left( 
N \log 2\pi\phi + 
{1\over{\phi}} 
\left( 
(\mathbf{y} - \mathbf{X} \beta)^T (\mathbf{y} - \mathbf{X} \beta) - 2(\mathbf{y} - \mathbf{X} \beta)^T \mu_N + \mathbf{1}^T_N (\mu_N^2+\mathbf{v}_N) 
\right) 
\\ + M \log 2\pi\psi +
\left( 
(\mathbf{u} \gamma)^T (\mathbf{u} \gamma) - 2(\mathbf{u} \gamma)^T\mu_M + \mathbf{1}^T_M (\mu_M^2+\mathbf{v}_M) 
\right) 
\right)\\
\end{aligned}
$$

where $\mu_M,\mathbf{v}_M$ are $(M\times 1)$ vectors of posterior means and variances, respectively, and $\mu_M,\mathbf{v}_M$ are their corresponding stacked versions with elements $\mu_j, v_j$ repeated $n_j$ times.

### M-step

Since $Q(\theta, \theta_{t-1})$ is just a quadratic form, we can maximise the expected CLL through analytical solutions to the first-order condition $\nabla_{\theta}Q(\theta, \theta_{t-1}) = \mathbf{0}$. It is straight-forward to show that

$$
\begin{aligned}
&& \beta^{\text{MLE}}&= (\mathbf{X}^T\mathbf{X})^{-1} (\mathbf{X}^T \mathbf{y} - \mathbf{X}^T \mu_N) \\
&& \phi^{\text{MLE}}&= {1\over{N}} \left( (\mathbf{y} - \mathbf{X} \beta^{\text{MLE}})^T (\mathbf{y} - \mathbf{X} \beta^{\text{MLE}}) - 2(\mathbf{y} - \mathbf{X} \beta^{\text{MLE}})^T \mu_N + \mathbf{1}^T_N (\mu_N^2+\mathbf{v}_N) \right) \\
&& \gamma^{\text{MLE}}&= (\mathbf{u}^T\mathbf{u})^{-1} \mathbf{u}^T \mu_M\\
&& \psi^{\text{MLE}}&= {1\over{M}} \left( (\mathbf{u} \gamma^{\text{MLE}})^T (\mathbf{u} \gamma^{\text{MLE}}) - 2(\mathbf{u} \gamma^{\text{MLE}})^T\mu_M + \mathbf{1}^T_M (\mu_M^2+\mathbf{v}_M)  \right) \\
\end{aligned}
$$

### Code

We shall now go through how the estimation of the multilevel regression model through EM can be programmed in R. As a first step we use a simple helper function that sets up the model: `multilevel_model` takes as inputs the $(N \times K)$ matrix of inidividual-level predictors `X`, the $(N \times 1)$ output vector `y`, the $(M \times L)$ matrix of group-level predictors `u` and an $(M \times 1)$ vector `n_j` that indicates group sizes. It returns a an object of class `multilevel_model`. (Using a class is not strictly necessary but will turn out to be convenient later as we shall see.)

In [5]:
multilevel_model <- function(
  X,
  y,
  u,
  n_j
) {
  # Group index:
  group <- rep.int(1:nrow(u), times=n_j)
  u <- matrix(rep.int(u, times=n_j))
  U <- cbind(1, unique(u))
  model <- list(
    X=X,
    y=y,
    u=u,
    U=U,
    group=group,
    n_j=n_j
  )
  class(model) <- "multilevel_model"
  return(model)
}
# Prepare model:
model <- multilevel_model(syn_data$data$X, syn_data$data$y, syn_data$data$u, syn_data$data$n_j)

With the model set up we can introduce the main `em` method that fits the model. If no initial guesses for the parameter vector $\theta$ are supplied, the algorithm will take random intial guesses excpet for $\beta_0$ which is set to the pooled OLS estimate. The algorithm then iteratively performs the E-step and M-step until convergence. The method finally returns an object of class `em_output` (again not strictly necessary but convenient).

In [22]:
## --------------------- EM algorithm: --------------------- ##
em.multilevel_model <- function(
  model,
  theta0=NULL,
  tol=1e-9,
  print_progress=T
) {
  # Initialization: ----
  if (is.null(theta0)) {
    theta0 <- list(
      beta = qr.solve(model$X, model$y), # initialize as pooled OLS
      phi = runif(1, 0, 100),
      gamma = matrix(rnorm(2)),
      psi = runif(1, 0, 100)
    )
  }
  converged <- FALSE # initialize convergence condition as false
  iter_count <- 1
  # Recursion: ----
  while (!converged) {
    if (print_progress) {
      print(iter_count)
    }
    # 1.) E-step: ----
    posterior_moments <- posterior(model, theta0) # returns and (M x 1) vectors of posteriors
    Q0 <- Q(model, theta0, posterior_moments) # to compare below
    # 2.) M-step: ----
    theta <- update_theta(model, theta0, posterior_moments)
    # Recalculate given MAP parameter estimates:
    Q1 <- Q(model, theta, posterior_moments)
    # Check for convergence:
    if (print_progress) {
      print(sprintf("Improvement of %0.2f",Q1-Q0))
    }
    converged <- Q1-Q0 < tol
    theta0 <- theta # new theta 0
    iter_count <- iter_count + 1
  }
  # Allocate and return output:
  output <- list(
    model = model,
    coefficients = theta,
    n_iter = iter_count - 1
  )
  class(output) <- "em_output"
  return(output)
}

em <- function(
  model,
  theta0=NULL,
  tol=1e-9,
  print_progress=T
) {
  UseMethod("em")
}

You will note that this function depends on a few helper functions: `posterior` - returns the first two moments of the posterior density of the latent variable -, `update_theta` - updates parameters according to the first-order conditions - and finally `Q` which compute the expected CLL for a given $\theta$ and posterior moments. 
These functions are methods that relate to the `multilevel_model` class we introdcued above, since they depend on the distribution we assume for the data and latent variables. Setting the code up this way is convenient since according to our assumption we can alter these methods and use the `em` method independently in the same way as before. Before running the `em` method we need to source the methods we just introduced.

##### `posterior`

In [11]:
posterior.multilevel_model <- function(
  model,
  theta
) {
  n_j <- model$n_j
  list2env(theta, envir = environment())
  # Posterior density ----
  var_posterior <- function(phi, psi, n_j) {
    v <- (n_j*phi^(-1) + psi^(-1))^(-1)
    return(v)
  }
  # Posterior mean ----
  mean_posterior <- function(phi, psi, n_j, v, mu_a, mu_b) {
    v * n_j * phi^(-1) * (mu_a) +
      v * psi^(-1) * (mu_b)
  }
  # Compute density ----
  p <- t(
    sapply(
      1:length(n_j),
      function(j) {
        y <- matrix(model$y[model$group==j])
        X <- matrix(model$X[model$group==j,], ncol = ncol(model$X))
        U <- matrix(model$U[j,], ncol = ncol(model$U))
        if (nrow(X) > 1) {
          mu_a <- mean(y) - colMeans(X) %*% beta
        } else {
          mu_a <- mean(y) - X %*% beta
        }
        mu_b <- unique(U) %*% gamma
        # Variance:
        v <- var_posterior(phi, psi, n_j[j])
        # Mean:
        mu <- mean_posterior(phi, psi, n_j[j], v, mu_a, mu_b)
        # Density:
        return(c(v=v, mu=mu))
      }
    )
  )
  return(p)
}

posterior <- function(model, theta) {
  UseMethod("posterior")
}

##### `update_theta`

In [12]:
update_theta.multilevel_model <- function(model, theta, p) {
  # Gather:
  n_j <- model$n_j
  list2env(theta, envir = environment())
  X <- model$X
  y <- model$y
  N <- nrow(y)
  M <- length(n_j)
  U <- model$U
  # Posterior moments:
  mu_M <- matrix(p[,"mu"])
  v_M <- matrix(p[,"v"])
  mu_N <- matrix(rep.int(mu_M, times=n_j))
  v_N <- matrix(rep.int(v_M, times=n_j))
  # beta: ----
  beta_map <- qr.solve(crossprod(X), (crossprod(X,y) - crossprod(X, mu_N)))
  theta$beta <- beta_map # update
  # phi: ----
  phi_map <- (1/N) *
    (crossprod(y-X%*%beta_map) - 2 * crossprod(y - X %*% beta_map, mu_N) + sum( mu_N^2 + v_N ))
  theta$phi <- phi_map # update phi
  # gamma: ----
  gamma_map <- qr.solve(crossprod(U), crossprod(U,mu_M))
  theta$gamma <- gamma_map # update gamma
  # psi: ----
  psi_map <- (1/M) *
    (crossprod(U %*% gamma_map) + sum( mu_M^2 + v_M ) - 2 * crossprod(U %*% gamma_map, mu_M))
  theta$psi <- psi_map # update psi
  return(theta)
}

update_theta <- function(model, theta, p) {
  UseMethod("update_theta")
}

##### `Q`

In [13]:
Q.multilevel_model <- function(model, theta, p) {
  n_j <- model$n_j
  list2env(theta, envir = environment())
  y <- model$y
  X <- model$X
  U <- model$U
  N <- nrow(y)
  M <- length(n_j)
  # Posterior moments:
  mu_M <- matrix(p[,"mu"])
  v_M <- matrix(p[,"v"])
  mu_N <- matrix(rep.int(mu_M, times=n_j))
  v_N <- matrix(rep.int(v_M, times=n_j))
  # Compute:
  A <- N * log(2*pi*phi)
  B <- phi^(-1) * ( crossprod(y - X %*% beta) - 2 * crossprod(y - X %*% beta, mu_N) + sum( mu_N^2 + v_N ) )
  C <- M * log(2*pi*psi)
  D <- psi^(-1) * ( crossprod(U %*% gamma) + sum( mu_M^2 + v_M ) - 2 * crossprod(U %*% gamma, mu_M))
  value <- (-1/2) * ( A + B + C + D )
  return(value)
}

Q <- function(model, theta, p) {
  UseMethod("Q")
}

In fact there is one further method relating to the `multilevel_model` which is used under the hood to infer the latent variables given our final estimate of $\theta$. The `infer_latent` method is not used anywhere inside the `em` method since the algorithm that fits the model need not know the estimated values of the latent factors at any point - it depends solely on estimates of their posterior moments. But estimated latent variables will be important in order to predict from the fitted model among other things, so we introduce the method that infers them here:

In [17]:
infer_latent.multilevel_model <- function(model, theta) {
  n_j <- model$n_j
  list2env(theta, envir = environment())
  z <- model$U %*% gamma
  return(z)
}

infer_latent <- function(model, theta) {
  UseMethod("infer_latent")
}

Finally let us briefly introduce a couple of standard methods relating the model output - i.e. the `em_output` object returned by the `em` method. We will leave these uncommented as they perform standard procedures.

In [21]:
## --------------------- Standard methods: --------------------- ##
predict.em_output <- function(em_output) {
  # Predictors: ----
  X <- em_output$model$X
  beta <- em_output$coefficients$beta
  # Latent variable estimates: ----
  Z_M <- infer_latent(em_output$model, em_output$coefficients)
  n_j <- em_output$model$n_j
  Z_N <- rep.int(Z_M, n_j)
  # Fit: ----
  fitted <- Z_N + X %*% beta
  return(fitted)
}

predict <- function(em_output) {
  UseMethod("predict")
}

residuals.em_output <- function(em_output) {
  y <- em_output$model$y
  fitted <- predict(em_output)
  res <- y - fitted
  return(res)
}

residuals <- function(em_output) {
  UseMethod("residuals")
}

coefficients.em_output <- function(em_output) {
  beta <- em_output$coefficients$beta
  if (!is.null(colnames(em_output$model$X))) {
    beta_names <- colnames(em_output$model$X)
  } else {
    beta_names <- sprintf("X_%i", 1:length(beta))
  }
  Z_M <- infer_latent(em_output$model, em_output$coefficients)
  Z_M_names <- sprintf("z_%i", 1:nrow(Z_M))
  coeffs <- data.table::data.table(
    Covariate = c(beta_names, Z_M_names),
    Coefficient = c(beta, Z_M)
  )
  return(coeffs)
}

coefficients <- function(em_output) {
  UseMethod("coefficients")
}

rsquared.em_output <- function(em_output) {
  y <- em_output$model$y
  fitted <- predict(em_output)
  rss <- sum((fitted - y)^2)
  tss <- sum((y-mean(y))^2)
  rsq <- 1 - rss/tss
  return(rsq)
}

rsquared <- function(em_output) {
  UseMethod("rsquared")
}

print.em_output <- function(em_output) {
  cat("Coefficient estimates: -----\n")
  print(coefficients(em_output))
  cat("Precision estimates: -----\n")
  cat(sprintf("Data: %0.2f\n",(em_output$coefficients$phi)^(-1)))
  cat(sprintf("Latent: %0.2f\n",(em_output$coefficients$psi)^(-1)))
  cat(sprintf("R-squared: %0.2f\n", rsquared(em_output)))
  cat("Performance: -----\n")
  cat(sprintf("Number of iterations: %0.2f\n", em_output$n_iter))
}

### First run

Finally, let us a perform a first run of our model. With the synthetic data and the corresponding `multilevel_model` already prepared we can simply run the following:

In [25]:
em(model, print_progress = F)

Coefficient estimates: -----
      Covariate Coefficient
   1:       X_1  1.00604683
   2:       X_2  0.09589592
   3:       z_1  0.71802461
   4:       z_2  0.88875274
   5:       z_3  1.81341163
  ---                      
 998:     z_996  0.96122204
 999:     z_997  1.56106945
1000:     z_998  0.30935784
1001:     z_999  0.73759356
1002:    z_1000  0.87892499
Precision estimates: -----
Data: 9.96
Latent: 19.76
R-squared: 0.90
Performance: -----
Number of iterations: 16.00

As per the output, the algorithm converged after 16 iterations. It attributes a precision to the group-level information roughly twice as high as the individual-level data, which was expected given the default parameter choices from above.

## Numerical test

## Inference

## Real data 

## Appendix